# regression for 2011-2018 target dates

Carry out regression experiment for a fixed set of predictors and all 2011-2018 target dates.

## Load packages

In [1]:
# Autoreload packages that are modified
%load_ext autoreload
%autoreload 2

# Plotting magic
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt

# Load relevant packages
import numpy as np
import pandas as pd
from sklearn import *
import sys
import subprocess
from datetime import datetime, timedelta
import netCDF4
import time
from functools import partial
import os

# Ensure that working directory is forecast_rodeo
if os.path.basename(os.getcwd()) == "experiments":
    # Navigate to forecast_rodeo
    os.chdir(os.path.join("..",".."))
if os.path.basename(os.getcwd()) != "forecast_rodeo":
    raise Exception("You must be in the forecast_rodeo folder")

# Adds 'experiments' folder to path to load experiments_util
sys.path.insert(0, 'src/experiments')
# Load general utility functions
from experiments_util import *
# Load functionality for fitting and predicting
from fit_and_predict import *
# Load functionality for evaluation
from skill import *
# Load stepwise utility functions
from stepwise_util import *

# Experiment name
experiment = "regression"

In [2]:
#
# Choose target
#
gt_id = "contest_tmp2m" # "contest_precip" or "contest_tmp2m"
target_horizon = "34w" # "34w" or "56w"

#
# Set variables based on target choice
#

# Identify measurement variable name
measurement_variable = get_measurement_variable(gt_id) # 'tmp2m' or 'precip'

# column names for gt_col, clim_col and anom_col 
gt_col = measurement_variable
clim_col = measurement_variable+"_clim"
anom_col = get_measurement_variable(gt_id)+"_anom" # 'tmp2m_anom' or 'precip_anom'

# anom_inv_std_col: column name of inverse standard deviation of anomalies for each start_date
anom_inv_std_col = anom_col+"_inv_std"

# Name of knn columns
knn_cols = ["knn"+str(ii) for ii in xrange(1,21)]

#
# Create list of official contest submission dates in YYYYMMDD format
#
submission_dates = [datetime(y,4,18)+timedelta(14*i) for y in range(2011,2018) for i in range(26)]
submission_dates = ['{}{:02d}{:02d}'.format(date.year, date.month, date.day) for date in submission_dates]
submission_dates = [datetime.strptime(str(d), "%Y%m%d") for d in submission_dates]
submission_dates = pd.Series(submission_dates)

#
# Create list of target dates corresponding to submission dates in YYYYMMDD format
#
target_dates = pd.Series([get_target_date('{}{:02d}{:02d}'.format(date.year, date.month, date.day), target_horizon) for date in submission_dates])

# Find all unique target day-month combinations
target_day_months = pd.DataFrame({'month' : target_dates.dt.month, 
                                  'day': target_dates.dt.day}).drop_duplicates()

In [3]:
#
# Choose regression parameters
#
# Record standard settings of these parameters
setting = "knn_regression"
if setting == "knn_regression":
    base_col = clim_col
    # Number of KNN neighbors to use in regression
    num_nbrs = 1 if gt_id.endswith('precip') else 20
    x_cols = ['knn'+str(nbr) for nbr in xrange(1,num_nbrs+1)] + ['ones']
    # Construct fixed lag anomaly variable names
    lags = (['43', '86'] if target_horizon == '56w' else ['29', '58']) + ['365']
    x_cols = x_cols + [measurement_variable+'_shift'+lag+'_anom' for lag in lags] 
    # Determine margin for local regression
    margin_in_days = 56 if gt_id.endswith('precip') else None
    # columns to group by when fitting regressions (a separate regression
    # is fit for each group); use ['ones'] to fit a single regression to all points
    group_by_cols = ['lat', 'lon']
    # anom_scale_col: multiply anom_col by this amount prior to prediction
    # (e.g., 'ones' or anom_inv_std_col)
    anom_scale_col = anom_inv_std_col
    # pred_anom_scale_col: multiply predicted anomalies by this amount
    # (e.g., 'ones' or anom_inv_std_col)
    pred_anom_scale_col = anom_scale_col
elif setting == "stepwise_no_model_selection":
    base_col = clim_col
    x_cols = default_stepwise_candidate_predictors(gt_id, target_horizon, hindcast=False) + ['knn1']
    margin_in_days = 56
    group_by_cols = ['lat', 'lon']
    anom_scale_col = anom_inv_std_col
    pred_anom_scale_col = anom_scale_col

print x_cols

['knn1', 'knn2', 'knn3', 'knn4', 'knn5', 'knn6', 'knn7', 'knn8', 'knn9', 'knn10', 'knn11', 'knn12', 'knn13', 'knn14', 'knn15', 'knn16', 'knn17', 'knn18', 'knn19', 'knn20', 'ones', 'tmp2m_shift29_anom', 'tmp2m_shift58_anom', 'tmp2m_shift365_anom']


In [6]:
#
# Default regression parameter values
#
# choose first year to use in training set
first_train_year = 1948 if gt_id == 'contest_precip' else 1979
# specify regression model
fit_intercept = False
model = linear_model.LinearRegression(fit_intercept=fit_intercept)

#
# Prepare target and feature data
#
relevant_cols = set(x_cols+[base_col,clim_col,anom_col,'sample_weight','target',
                    'start_date','lat','lon','year','ones']+group_by_cols)
# Create dataset with relevant columns only; otherwise the dataframe is too big
tic()
print "Loading date and lat lon date data"
data_dir = os.path.join("results", experiment, "shared", gt_id + "_" + target_horizon)
date_data = pd.read_hdf(os.path.join(data_dir, "date_data-" + gt_id + "_" + target_horizon + ".h5"))
date_data['year'] = date_data.start_date.dt.year
lat_lon_date_data = pd.read_hdf(
    os.path.join(data_dir, "lat_lon_date_data-" + gt_id + "_" + target_horizon + ".h5"))
toc()
# Restrict data to years >= first_train_year
tic()
print "Restricting data to years >= {}".format(first_train_year)
lat_lon_date_data = lat_lon_date_data.loc[lat_lon_date_data.start_date.dt.year >= first_train_year]
date_data = date_data.loc[date_data.year >= first_train_year]
toc()
# Drop rows with missing values for any relevant column 
tic()
print "Dropping rows with missing values for any relevant columns"
relevant_lat_lon_date_cols = list(set(lat_lon_date_data.columns.tolist()) & relevant_cols)
lat_lon_date_data.dropna(subset = relevant_lat_lon_date_cols, inplace = True)
relevant_date_cols = list(set(date_data.columns.tolist()) & relevant_cols)
date_data.dropna(subset = relevant_date_cols, inplace = True)
toc()
# Add supplementary columns
tic()
print "Adding supplementary columns"
lat_lon_date_data['anom_inv_sqrt_2nd_mom'] = 1.0/np.sqrt(
    lat_lon_date_data.groupby('start_date')[anom_col].transform('mean')**2
    + lat_lon_date_data.groupby('start_date')[anom_col].transform('var',ddof=0))
lat_lon_date_data['ones'] = 1.0
lat_lon_date_data['zeros'] = 0.0
# To minimize the mean-squared error between predictions of the form
# (f(x_cols) + base_col - clim_col) * pred_anom_scale_col
# and a target of the form anom_col * anom_scale_col, we will
# estimate f using weighted least squares with datapoint weights
# pred_anom_scale_col^2 and effective target variable 
# anom_col * anom_scale_col / pred_anom_scale_col + clim_col - base_col
lat_lon_date_data['sample_weight'] = lat_lon_date_data[pred_anom_scale_col]**2
lat_lon_date_data['target'] = (lat_lon_date_data[clim_col] - lat_lon_date_data[base_col] + 
                               lat_lon_date_data[anom_col] * lat_lon_date_data[anom_scale_col] / 
                              (lat_lon_date_data[pred_anom_scale_col]+(lat_lon_date_data[pred_anom_scale_col]==0)))
toc()

# Load KNN data
tic()
print "Loading KNN data"
past_days = 60
days_early = 337 if target_horizon == "34w" else 323
max_nbrs = 20
knn_dir = os.path.join("data", "dataframes")
knn_data = pd.read_hdf(
    os.path.join(knn_dir, 
                 "knn-{}-{}-days{}-early{}-maxnbrs{}.h5".format(
                     gt_id, target_horizon, past_days, days_early, max_nbrs)))
relevant_knn_cols = list(set(knn_cols) & relevant_cols)
# Divide knn anomalies by std dev across grid cells
knn_data[relevant_knn_cols] /= knn_data.groupby(["start_date"])[relevant_knn_cols].transform('std')
toc()

# Restrict data to relevant columns
tic()
print "Merge datasets"
relevant_lat_lon_date_cols = list(set(lat_lon_date_data.columns.tolist()) & relevant_cols)
data = lat_lon_date_data.loc[:, relevant_lat_lon_date_cols]
relevant_knn_cols = list(set(knn_data.columns.tolist()) & relevant_cols)
data = pd.merge(data, knn_data[relevant_knn_cols],
                on=["start_date","lat","lon"], how="left")
data = pd.merge(data, date_data[relevant_date_cols],
                on="start_date", how="left")
print data.head()
del lat_lon_date_data
del knn_data
del date_data
toc()

# Print warning if not all x columns were included
s = [x for x in x_cols if x not in data.columns.tolist()]
if s:
    print "These x columns were not found:"
    print s

Loading date and lat lon date data
Elapsed time: 7.014532 seconds.

Restricting data to years >= 1979
Elapsed time: 7.854232 seconds.

Dropping rows with missing values for any relevant columns
Elapsed time: 15.536588 seconds.

Adding supplementary columns
Elapsed time: 14.896080 seconds.

Loading KNN data
Elapsed time: 29.981443 seconds.

Merge datasets
   tmp2m_shift29_anom  tmp2m_shift365_anom    target  tmp2m_anom    lon  \
0           -1.017055            -6.193481  1.045174    1.045174  261.0   
1           -1.011894            -6.533477  1.645593    1.645593  261.0   
2           -1.320754            -6.079782  2.102227    2.102227  261.0   
3           -1.537798            -4.923485  2.490746    2.490746  261.0   
4           -1.715973            -4.072896  3.116631    3.116631  261.0   

   tmp2m_clim  tmp2m_shift58_anom  ones   lat  sample_weight  ...  knn9  knn8  \
0   13.877076           -1.931445   1.0  27.0       0.193277  ...   NaN   NaN   
1   13.817056           -1.785

In [7]:
num_cores = 8
# For the target (day, month) combination, fit leave-one-year regression model 
# on training set subsetted to relevant margin and generate predictions for each 
# held-out year
prediction_func = rolling_linear_regression_wrapper
all_preds = pd.DataFrame()

generic_year = 2011
for day_month in target_day_months.itertuples():
    # Get target date on generic as a datetime object
    target_date_obj = datetime.strptime('{}{:02d}{:02d}'.format(
        generic_year, day_month.month, day_month.day), "%Y%m%d")
    print 'day_month={}, target={}'.format(day_month, target_date_obj)
    # Get number of days between start date of observation period used for prediction
    # (2 weeks ahead) and start date of target period (2 or 4 weeks ahead) + 1 day do 
    # to practical constraints of submission
    start_delta = get_start_delta(target_horizon) # 29 or 43
    # Create template for held-out years: each held-out year will run from 
    # last_train_date + 1 on that year (inclusive) through last_train_date
    # of the next year (inclusive)
    last_train_date = target_date_obj - timedelta(start_delta)
    if margin_in_days is not None:
        tic()
        sub_data = month_day_subset(data, target_date_obj, margin_in_days)
        toc()
    else:
        sub_data = data
    tic()
    preds = apply_parallel(sub_data.groupby(group_by_cols),
                           prediction_func, num_cores,
                           x_cols=x_cols, 
                           base_col=base_col, 
                           clim_col=clim_col, 
                           anom_col=anom_col, 
                           last_train_date=last_train_date)
    preds = preds.reset_index() 
    # Only keep the predictions from the target day and month
    preds = preds[(preds.start_date.dt.day == target_date_obj.day) & 
                  (preds.start_date.dt.month == target_date_obj.month)]
    # Concatenate predictions
    all_preds = pd.concat([all_preds, preds])
    toc()
    #---------------
    # Evaluate only on target dates                                                        
    #---------------
    tic()
    skills = get_col_skill(
        all_preds[all_preds.start_date.isin(target_dates)], 
        "truth", "forecast", time_average = False)
    print "running mean skill = {}".format(skills.mean())
    toc()

day_month=Pandas(Index=0, day=2, month=5), target=2011-05-02 00:00:00
Elapsed time: 110.634363 seconds.

running mean skill = 0.226607041983
Elapsed time: 0.046922 seconds.

day_month=Pandas(Index=1, day=16, month=5), target=2011-05-16 00:00:00
Elapsed time: 108.636911 seconds.

running mean skill = 0.141737205833
Elapsed time: 0.067178 seconds.

day_month=Pandas(Index=2, day=30, month=5), target=2011-05-30 00:00:00
Elapsed time: 108.508837 seconds.

running mean skill = 0.288329436148
Elapsed time: 0.097602 seconds.

day_month=Pandas(Index=3, day=13, month=6), target=2011-06-13 00:00:00
Elapsed time: 109.159429 seconds.

running mean skill = 0.306017968223
Elapsed time: 0.114264 seconds.

day_month=Pandas(Index=4, day=27, month=6), target=2011-06-27 00:00:00
Elapsed time: 112.608668 seconds.

running mean skill = 0.380045582218
Elapsed time: 0.160544 seconds.

day_month=Pandas(Index=5, day=11, month=7), target=2011-07-11 00:00:00
Elapsed time: 105.847093 seconds.

running mean skill =

In [10]:
target_day_months

,day,month
0,2,5
1,16,5
2,30,5
3,13,6
4,27,6
5,11,7
6,25,7
7,8,8
8,22,8
9,5,9


In [8]:
#---------------
# Evaluate on all target dates                                                        
#---------------
tic()
skills = get_col_skill(
    all_preds[all_preds.start_date.isin(target_dates)], 
    "truth", "forecast", time_average = False)
print "overall mean skill = {}".format(skills.mean())
#---------------
# Evaluate on contest year                                                      
#---------------
skills = get_col_skill(
    all_preds[all_preds.start_date.isin(target_dates) & (all_preds.start_date >= 
              get_target_date("20170418", target_horizon))], 
    "truth", "forecast", time_average = False)
print "2017-2018 mean skill = {}".format(skills.mean())
toc()

overall mean skill = 0.311124936145
2017-2018 mean skill = 0.280716584179
Elapsed time: 0.614542 seconds.



In [9]:
#
# Save target date predictions to file
#

# Ensure hash randomization turned off for reproducibility
PYTHONHASHSEED=0
# Define a compact string identifier for experiment parameters
param_str = str(abs(hash((base_col, frozenset(x_cols), margin_in_days, frozenset(group_by_cols),
                         anom_scale_col, pred_anom_scale_col))))
# Create directory for storing results
outdir = os.path.join('results',experiment,'2011-2018',gt_id+'_'+target_horizon,param_str)
if not os.path.exists(outdir):
    os.makedirs(outdir)
# Write predictions to file; record the dependency on the type of knn features integrated
# into the feature set in the file name
preds_file = os.path.join(
    outdir,'preds-{}-{}-days{}-early{}.h5'.format(gt_id,target_horizon,past_days,days_early))
print "Saving predictions to "+preds_file; tic()
all_preds[all_preds.start_date.isin(target_dates)].to_hdf(preds_file, key="data", mode="w"); toc()

Saving predictions to results/regression/2011-2018/contest_tmp2m_34w/3398913618608284965/preds-contest_tmp2m-34w-days60-early337.h5
Elapsed time: 0.192340 seconds.

